# OPEN COESIONE Indagine Trasparenza

## Data Wrangling sui dataset pubblicati



In [7]:
import pandas as pd
import glob
import os

In [8]:
#Path con i dataset provenienti da zip pubblicato sul sito
path = "../data/raw/indagine_trasparenza/"
all_files = glob.glob(path + "/*.csv")

#Creo dictionary con chiave nome dataframe e value il dataframe
dict_ = {}

for file in all_files:
    fname = os.path.basename(file)
    dict_[fname] = pd.read_csv(file, sep=";")


### NOTE:

- Non c'è il dataset relativo al 2016 e al 2018
- I dataset contengono un numero differente di colonne
- La rilevazione è stata con frequenza **annuale** fino al 2015

- I dataset fino al 2015 hanno una entry/riga per **ogni stato** e per **ogni programma**

- I dataset a partire dalla rilevazione successiva (non più annuale ma ogni due anni) hanno una entry/riga per **ogni stato** (non più per ogni programma poichè a partire dall’anno 2016/17 c’è l’obbligo di avere un unico portale quindi le rilevazioni vengono fatte in un unico punto per tutti i programmi.)



### TODO:

Nel campo "PAESE" ci sono:
- record con nome del paese nella sua lingua d'origine e alcuni valori
- alcuni recordi che è necessario capire come trattare, sei record con:

    EU Interregional Co-operation and Networking Programmes 
    
    EU cross-border cooperation 
    
    EU trasnational cooperation 

Per ogni dataset:

- Cercare dizionario stati UE
- Aggiungere colonna codifica con lo stato 
- Cercare dizionario denominazione inglese dei programmi (tabella di lookup)
- Integrare denominazione programma
- Attenzione a encoding - UTF 8


In [9]:
from collections import Counter
paesi = []
for df in dict_:
    paesi.extend(dict_[df].PAESE.unique())

In [16]:
#API Search by native name
#https://restcountries.eu/#filter-response
#example: https://restcountries.eu/rest/v2/name/italia?
#example: https://restcountries.eu/rest/v2/name/italia?fields=name;alpha2Code;alpha3Code
        
import requests

url = 'https://restcountries.eu/rest/v2/name/'
#voglio il nome in inglese, il codice ISO a due e tre caratteri
parameter= '?fields=name;alpha2Code;alpha3Code'

In [11]:
#creo dataframe con la lista dei valori dei paesi
countries = []

for native,v in Counter(paesi).items():
    #concateno request con parametri
    req_url = url + native + parameter
    
    response = requests.get(req_url)
    # trace per quelli non trovati
    if response.status_code == 200:
        data = response.json()
        # concateno in lista l'origin e i parametri in risposta
        countries.append([native, data[0].get("name"), data[0].get("alpha2Code"), data[0].get("alpha3Code")])
    #creo dataframe con le liste per ogni risposta    
    df_countries = pd.DataFrame(countries)

In [12]:
# set column names
df_countries.columns = ['PAESE', 'EN_PAESE', 'alpha2Code', 'alpha3Code']

In [13]:
df_countries

,PAESE,EN_PAESE,alpha2Code,alpha3Code
0,Ceska Republika,Czech Republic,CZ,CZE
1,Croatia,Croatia,HR,HRV
2,Danmark,Denmark,DK,DNK
3,Deutschland,Germany,DE,DEU
4,Eesti,Estonia,EE,EST
5,Ellada,Greece,GR,GRC
6,España,Spain,ES,ESP
7,France,France,FR,FRA
8,Ireland,Ireland,IE,IRL
9,Italia,Italy,IT,ITA


## Aggiunta al dataset campi su nazione

fix Balgarija

In [14]:
for df in dict_:
    #
    
    df_app = df
    result = pd.merge(dict_[df],
                 df_countries,
                 on='PAESE',
                 how="left")
    
    result.to_csv("../data/processed/"+df, index=None)

In [15]:
result

,PAESE,PRES_ASS_PAG,PRES_OC_DATA_FINE_PREVISTA,PRES_OC_DATA_INIZIO_PREVISTA,PRES_STATUS,PRES_FINANZ_ALTRO,PRES_FINANZ_UE,PRES_IMPEGNI,PRES_TOT_PAGAMENTI,PRES_CAP_PROG,...,IN_DOC,IND_XML,BANDI_OPP,CONS_PUBB_PO,CONS_PUBB_STRATEGIE,CONS_PUBB_CALL,BEST_PRAC,EN_PAESE,alpha2Code,alpha3Code
0,Balgarija,0,1,1,1,1,1,1,0,0,...,0,1,1.0,0.0,0.0,0.0,0,NaN,NaN,NaN
1,Belgique-België,0,1,1,0,0,1,0,0,1,...,0,0,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN
2,Ceska Republika,1,1,1,1,1,1,1,1,1,...,0,0,1.0,0.0,0.0,0.0,0,Czech Republic,CZ,CZE
3,Croazia,0,1,1,0,0,1,1,0,1,...,0,0,1.0,1.0,0.0,0.0,0,NaN,NaN,NaN
4,Danmark,0,1,1,0,0,1,1,0,1,...,0,0,1.0,0.0,0.0,0.0,0,Denmark,DK,DNK
5,Deutschland,0,1,1,0,0,1,1,0,1,...,0,0,1.0,0.0,0.0,0.0,0,Germany,DE,DEU
6,Eesti,0,1,1,1,0,1,1,0,0,...,0,0,1.0,0.0,0.0,0.0,1,Estonia,EE,EST
7,Ellada,1,1,1,0,0,1,1,0,1,...,0,0,1.0,0.0,0.0,0.0,0,Greece,GR,GRC
8,España,0,1,1,0,0,1,1,0,1,...,0,0,NaN,0.0,0.0,0.0,1,Spain,ES,ESP
9,France,1,1,1,0,0,1,1,0,1,...,0,0,0.0,1.0,0.0,0.0,1,France,FR,FRA
